# FCT Unesp - TCC Treino 2
Aluno: Pétrus Antonio Bárbosa Pradella

TCC2 - Treinando análise preditiva para dados de transações comerciais.

Teste 1
    - DataSet Utilizado: BigMart Sales Data
    - Fonte: Kaggle
    - Link: https://www.kaggle.com/brijbhushannanda1979/bigmart-sales-data
    
Caracteristicas gerais do Dataset:

    - Número de Instancias: 14204
    - Número de Atributos: 11
    - Estatisticas de Uso em 18/09/2019 (kaggle):
        23,852 views
        5,934 downloads


### Importando as bibliotecas necessárias

In [1]:
from myownapi.AnalyticsLR import AnalyticsLR;

analytics = AnalyticsLR();

### Carregando a base de dados

In [2]:
onlineMode = False;

if onlineMode:
    dataset_folder_url = "http://downloads.finalcraft.com.br/tcc/BigMartSalesData";
else:
    dataset_folder_url = "C:/Users/Petrus/Desktop/UNESP/Docs 2016-2020/2019/Segundo Semestre/TCC2/TCC BigData Analytics/Treino6";

dataset_file_train = dataset_folder_url + '/Train.csv';
dataset_file_test = dataset_folder_url + '/Test.csv';

dataset_train = analytics.read_csv(dataset_file_train);
dataset_test = analytics.read_csv(dataset_file_test);

import pandas as pd;
result = pd.concat([dataset_train,dataset_test], sort=True);

analytics.setDataframe(result);

analytics.df.head(5)

Iniciando leitura do arquivo:
 --> C:/Users/Petrus/Desktop/UNESP/Docs 2016-2020/2019/Segundo Semestre/TCC2/TCC BigData Analytics/Treino6/Train.csv
Arquivo lido com sucesso!
Iniciando leitura do arquivo:
 --> C:/Users/Petrus/Desktop/UNESP/Docs 2016-2020/2019/Segundo Semestre/TCC2/TCC BigData Analytics/Treino6/Test.csv
Arquivo lido com sucesso!


,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type
0,Low Fat,FDA15,249.8092,3735.1380,Dairy,0.016047,9.30,1999,OUT049,Tier 1,Medium,Supermarket Type1
1,Regular,DRC01,48.2692,443.4228,Soft Drinks,0.019278,5.92,2009,OUT018,Tier 3,Medium,Supermarket Type2
2,Low Fat,FDN15,141.6180,2097.2700,Meat,0.016760,17.50,1999,OUT049,Tier 1,Medium,Supermarket Type1
3,Regular,FDX07,182.0950,732.3800,Fruits and Vegetables,0.000000,19.20,1998,OUT010,Tier 3,NaN,Grocery Store
4,Low Fat,NCD19,53.8614,994.7052,Household,0.000000,8.93,1987,OUT013,Tier 3,High,Supermarket Type1


### Filtrando os atributos Normalizados

In [3]:
analytics.columns()


Index(['Item_Fat_Content', 'Item_Identifier', 'Item_MRP', 'Item_Outlet_Sales',
       'Item_Type', 'Item_Visibility', 'Item_Weight',
       'Outlet_Establishment_Year', 'Outlet_Identifier',
       'Outlet_Location_Type', 'Outlet_Size', 'Outlet_Type'],
      dtype='object')

In [ ]:
analytics.

In [ ]:
#n_produtos = dataset_filtered[['Product_Code']].count().sum();  # Dá erro pois Product_code virou um index_de_coluna e não mais uma coluna separada!

n_produtos = dataset_filtered.shape[0]; #Como fizemos a removeção da coluna de Product_code não dá mais para contar ela, contudo o número de produtos é o mesmo que o número de linhas!
n_semanas = len(dataset_filtered.columns);  #Se não tivesse removido a primeira coluna, deveria ser [nColumns - 1];

print("Número de Produtos Cadastrados: ", n_produtos)
print("Número de Semanas Contabilizadas: ", n_semanas)


### Aplicando lógica de regressão linear! (Ténicas usada "Spearman")

In [ ]:
#Aplicando o método Spearman
# Mais informações Tecnicas: https://kite.com/python/examples/665/scipy-compute-the-spearman-correlation
# Mais informações Pedagógicas: https://statistics.laerd.com/statistical-guides/spearmans-rank-order-correlation-statistical-guide.php

#A ténicas Spearman é recomendada para esse caso pois trata-se de um regressão linear para um dataset que possui apenas dois atributos a serem analisados.

correlacao_produtos = dataset_filtered.transpose().corr(method='spearman')
print(correlacao_produtos.shape)
correlacao_produtos.head(5)



### Procurando valores com coeficiente alto!


In [ ]:
set_de_coeficiente_alto = set() # record the individual Product_Code values involved
pares_de_coeficiente_alto = [] # record the Product_Code pairs and the correlation value

for row in correlacao_produtos:
    for col in correlacao_produtos.columns:
        if row < col:
            
            valor_de_correlacao = correlacao_produtos.loc[row, col]

            if abs(valor_de_correlacao) >= limiar_de_correlacao:
                # Se o valor é maior que o limiar, salvar ele!
                #print(row)
                #print(col)
                #print(corr_val)
                pares_de_coeficiente_alto.append((row, col, valor_de_correlacao))
                #Adiciona a linha e a coluna
                set_de_coeficiente_alto.add(row) 
                set_de_coeficiente_alto.add(col)
            
#print(high_corr_list)
# Convert list to a dataframe
high_correlation_dataframe = pd.DataFrame(pares_de_coeficiente_alto, columns=['Product_1', 'Product_2', 'Corr'])
print(f"Número de produtos com coeficientes de correlação >= {limiar_de_correlacao} ou <= -{limiar_de_correlacao}: {high_correlation_dataframe.shape[0]}")
print(f"Número total de produtos: {len(set_de_coeficiente_alto)}")
high_correlation_dataframe.head(5)

### Analisando popularidade dos itens (maior e menor venda)


In [ ]:
n_produtos = dataset_filtered.shape[0];
n_semanas = len(dataset_filtered.columns);

print("Número de Produtos Cadastrados: ", n_produtos)
print("Número de Semanas Contabilizadas: ", n_semanas)
print(' ')
print(' ')

num_top = math.floor(param_top_perc * n_produtos)
print(f'Encontrar os [{num_top}] produtos mais e menos populares com base na média das vendas semanáis:')

media_venda_mensal = dataset_filtered.mean(axis=1)

produtos_mais_vendidos = media_venda_mensal.nlargest(n=num_top).index.values.tolist()
produtos_menos_vendidos = media_venda_mensal.nsmallest(n=num_top).index.values.tolist()

dataset_mais_vendidos = dataset_filtered[dataset_filtered.index.isin(produtos_mais_vendidos)]
dataset_menos_vendidos = dataset_filtered[dataset_filtered.index.isin(produtos_menos_vendidos)]

In [2]:

print(f"Foram encontrados os [{num_top}] produtos mais populares: {dataset_mais_vendidos.shape[0]}")
dataset_mais_vendidos.head(5)

NameError: name 'num_top' is not defined

In [ ]:

print(f"Foram encontrados os [{num_top}] produtos menos populares: {dataset_menos_vendidos.shape[0]}")
dataset_menos_vendidos.head(5)